In [1]:
!pip install xgboost

In [20]:
import boto3
import pandas as pd
import shutil
import os
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
import datetime
from pathlib import Path
from logging import INFO, basicConfig, info
import warnings
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [21]:
# setup logging
basicConfig(level=INFO, format='%(asctime)s\t%(levelname)s\t%(filename)s\t%(message)s')
warnings.filterwarnings('ignore', category=FutureWarning)  # ignore FutureWarning from scikit learn

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_info_columns = 1000

In [52]:
# load the dataset and set the date column
df = pd.read_csv('temp/01-data.csv', keep_default_na=False, na_values='')
df['date'] = pd.to_datetime(df['date'])



/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (98) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Load the model and scalers

In [62]:
from pickle import load

scalers = load(open('temp/scalers.pkl', 'rb'))
predictor = xgb.Booster({'nthread': 1})
predictor.load_model('temp/predictor.model')

# Start Predicting

In [107]:
# note, we can't handle future dates
days_to_predict = 10
for geo_code in df['geo_code'].unique():
    print(geo_code)
    df_geo = pd.read_csv(f"temp/prediction-baseline/prediction-baseline-{geo_code.replace('/', '-')}.csv", keep_default_na=False, na_values='')
    df_geo['date'] = pd.to_datetime(df['date'])
    df_geo = df_geo.sort_values(['date'])
    
    # filter the date range for the prediction window
    date_to_predict_from = df[df['predicted'] == False]['date'].max()
    date_to_predict_to = date_to_predict_from + pd.to_timedelta(days_to_predict, unit='d')   
    df_geo = df_geo.loc[df_geo['date'] >= date_to_predict_from]
    df_geo = df_geo.loc[df_geo['date'] <= date_to_predict_to]
    df_geo = df_geo.drop(labels=['date'], axis=1)

    # predict forward
    for idx, row in df_geo.iterrows():
        data = xgb.DMatrix(row.values.reshape(1,-1))
        v = predictor.predict(data)
        print(v)
    


AD


ValueError: feature_names mismatch: ['404'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', '